# Prepare 3D Object Detection Library
* Clone TensorFlow 3D
* Fix needed version of its dependencies
* Install other needed dependencies

In [ ]:
!ls
!pwd
!ls /tmp/

In [ ]:
# version tensorflow not defined in file _pywrap_tensorflow_internal.so with link time reference
# wrong tensorflow version, make sure using print(tf.__version__) that version is 2.3.0
# Restart and clear cell outputs first
import tensorflow as tf
print(tf.__version__)
import tensorflow_sparse_conv_ops as sparse_conv_ops

In [ ]:
#!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/x86_64-linux-gnu/ && echo $LD_LIBRARY_PATH 

In [ ]:
!head -50 google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh

In [2]:
# comment the tf3d semantic segmentation imports
!sed -e '/from tf3d.semantic_segmentation/s/^/#/g' -i google-research/tf3d/gin_imports.py
#!sed -i '/^DATASET_PATH/s/=.*$/=\/root\/waymo-night-training-v2\//' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh
!sed -i '/^DATASET_PATH/s/=.*$/=\/root\/waymo-mv-night-training\//' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh
#!sed -i 's/TRAIN_SPLIT="train"/TRAIN_SPLIT="filtered-night-train"/g' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh
!sed -i 's/TRAIN_SPLIT="train"/TRAIN_SPLIT="filtered-night-mv-train"/g' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh

!sed -i 's/NUM_GPUS=8/NUM_GPUS=1/g' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh
#!sed -i 's/NUM_EPOCHS=240/NUM_EPOCHS=10/g' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh
!sed -i 's/LOG_FREQ=100/LOG_FREQ=50/g' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh
!sed -i 's/NUM_STEPS_PER_EPOCH=100/NUM_STEPS_PER_EPOCH=321/g' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh
!sed -i 's/BATCH_SIZE=4/BATCH_SIZE=8/g' google-research/tf3d/object_detection/scripts/waymo/run_train_locally.sh

#### Fix a bug in example_parser.py ####
# features.deserialize_example(serialized_example) in example_parser.py:32, AttributeError: 'FeaturesDict' object has no attribute 'deserialize_example'
#!sed -i 's/deserialize_example/decode_example/g' google-research/tf3d/datasets/utils/example_parser.py
# revert commit 780a7a38
!sed -i 's/return features.serialize_example(example_data)/encoded = features.encode_example(example_data)'"\n"'  example_specs = features.get_serialized_info()'"\n"'  serializer = tfds.core.example_serializer.ExampleSerializer(example_specs)'"\n"'  return serializer.serialize_example(encoded)/g' google-research/tf3d/datasets/utils/example_parser.py
!sed -i 's/return features.deserialize_example(serialized_example)/example_specs = features.get_serialized_info()'"\n"'  parser = tfds.core.example_parser.ExampleParser(example_specs)'"\n"'  tfexample_data = parser.parse_example(serialized_example)'"\n"'  return features.decode_example(tfexample_data)/g' google-research/tf3d/datasets/utils/example_parser.py
#!sed -i.bak $'s/return features.serialize_example(example_data)/encoded = features.encode_example(example_data)\\\n  example_specs = features.get_serialized_info()\\\n  serializer = tfds.core.example_serializer.ExampleSerializer(example_specs)\\\n  return serializer.serialize_example(encoded)/g' google-research/tf3d/datasets/utils/example_parser.py
#!sed -i.bak $'s/return features.deserialize_example(serialized_example)/  example_specs = features.get_serialized_info()\\\n  parser = tfds.core.example_parser.ExampleParser(example_specs)\\\n  tfexample_data = parser.parse_example(serialized_example)\\\n  return features.decode_example(tfexample_data)/g' google-research/tf3d/datasets/utils/example_parser.py
#!cd /kaggle/working/google-research/tf3d/datasets/utils/ && git checkout example_parser.py
!cd google-research && export LD_LIBRARY_PATH=/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/opt/conda/lib:/usr/local/cuda-10.1/targets/x86_64-linux/lib/ && bash tf3d/object_detection/scripts/waymo/run_train_locally.sh 2>&1 | tee log3.txt

Deleting TRAIN_DIR at /tmp/tf3d_experiment/det_waymo_001...
2025-12-06 23:13:43.813958: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
I1206 23:13:46.103450 126745994245952 train.py:130] TF_CONFIG is Empty...
2025-12-06 23:13:46.106344: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2025-12-06 23:13:46.140288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-12-06 23:13:46.140664: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.73GiB deviceMemoryBandwidth: 836.37GiB/s
2025-12-06 23:13:46.140713: I tensorflow/stream_executor/platform/default

I1206 23:13:57.756376 126745994245952 api.py:603] view indices 2d inputs
I1206 23:13:57.765738 126745994245952 api.py:603] {'rgb_view': <tf.Tensor 'stack_6:0' shape=(5, None, 2) dtype=int32>}
I1206 23:13:57.775194 126745994245952 api.py:603] mesh inputs
I1206 23:13:57.784490 126745994245952 api.py:603] {'point_positions': <tf.Tensor 'concat:0' shape=(None, 3) dtype=float32>, 'point_intensities': <tf.Tensor 'concat_1:0' shape=(None, 1) dtype=float32>, 'point_elongations': <tf.Tensor 'concat_2:0' shape=(None, 1) dtype=float32>, 'point_normals': <tf.Tensor 'concat_3:0' shape=(None, 3) dtype=float32>}
I1206 23:13:57.794056 126745994245952 api.py:603] object inputs
I1206 23:13:57.803695 126745994245952 api.py:603] {'objects_rotation_matrix': <tf.Tensor 'boolean_mask_35/GatherV2:0' shape=(None, 3, 3) dtype=float32>, 'objects_center': <tf.Tensor 'boolean_mask_34/GatherV2:0' shape=(None, 3) dtype=float32>, 'objects_length': <tf.Tensor 'boolean_mask_31/GatherV2:0' shape=(None, 1) dtype=float32>

KeyboardInterrupt: 

In [1]:
!nvidia-smi
!echo "================ CUDA Version on Docker"
!nvcc --version
!cat /usr/local/cuda/version.txt


Sat Dec  6 23:13:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:09:00.0 Off |                    0 |
| N/A   39C    P0             43W /  300W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# backup_model  model  train
!ls /tmp/tf3d_experiment/det_waymo_001
!ls /tmp/tf3d_experiment/det_waymo_001/train

# Evaluate the Model

In [6]:
!head -50 google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh

# Copyright 2021 The Google Research Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#!/bin/bash

VERSION=001
JOB_NAME="det_waymo_${VERSION}"
EVAL_DIR="/tmp/tf3d_experiment/${JOB_NAME}"
CKPT_DIR="${EVAL_DIR}/model/"

NUM_STEPS_PER_EPOCH=246
LOG_FREQ=100

# Data
DATASET_NAME='waymo_object_per_frame'
EVAL_SPLIT="filtered-night-sf-val"
DATASET_PATH=/root/waymo-sf-night-validation/

# Gin config
IMPORT_MODULE='tf3d.gin_imports'
EVAL_GIN_CONFIG="tf3d/object_detection/configs/waymo_eval.gin"



In [ ]:

#!sed -i '/^DATASET_PATH/s/=.*$/=\/root\/waymo-night-validation-dataset\//' google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh
!sed -i '/^DATASET_PATH/s/=.*$/=\/root\/waymo-sf-night-validation\//' google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh
#!sed -i '/^DATASET_PATH/s/=.*$/=\/root\/waymo-mv-night-training\//' google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh
!sed -i '/^EVAL_SPLIT/s/=.*$/="filtered-night-sf-val"/' google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh
#!sed -i 's/EVAL_SPLIT=\'val\'/EVAL_SPLIT=\'filtered-night-sf-val\''/g' google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh

#!sed -i 's/\/tmp\/tf3d_experiment\/${JOB_NAME}/\/kaggle\/working\/${JOB_NAME}/g' google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh

!sed -i 's/NUM_STEPS_PER_EPOCH=100/NUM_STEPS_PER_EPOCH=25/g' google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh
#!sed -i 's/LOG_FREQ=100/LOG_FREQ=10/g' google-research/tf3d/object_detection/scripts/waymo/run_eval_locally.sh

!cd google-research && bash tf3d/object_detection/scripts/waymo/run_eval_locally.sh 2>&1 | tee val_log3.txt

EVAL_DIR at /tmp/tf3d_experiment/det_waymo_001...
2025-12-07 19:47:56.114794: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2025-12-07 19:47:59.350361: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2025-12-07 19:47:59.358485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-12-07 19:47:59.358798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.73GiB deviceMemoryBandwidth: 836.37GiB/s
2025-12-07 19:47:59.358836: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2025-12-07 

I1207 19:48:13.389085 123335559104320 api.py:603] {'objects_rotation_matrix': <tf.Tensor 'boolean_mask_35/GatherV2:0' shape=(None, 3, 3) dtype=float32>, 'objects_center': <tf.Tensor 'boolean_mask_34/GatherV2:0' shape=(None, 3) dtype=float32>, 'objects_length': <tf.Tensor 'boolean_mask_31/GatherV2:0' shape=(None, 1) dtype=float32>, 'objects_width': <tf.Tensor 'boolean_mask_33/GatherV2:0' shape=(None, 1) dtype=float32>, 'objects_height': <tf.Tensor 'boolean_mask_32/GatherV2:0' shape=(None, 1) dtype=float32>, 'objects_class': <tf.Tensor 'boolean_mask_36/GatherV2:0' shape=(None, 1) dtype=int32>}
I1207 19:48:13.398523 123335559104320 api.py:603] non_tensor_inputs
I1207 19:48:13.407590 123335559104320 api.py:603] {'camera_intrinsics': <tf.Tensor 'Reshape_94:0' shape=(3, 3) dtype=float32>, 'camera_rotation_matrix': <tf.Tensor 'Reshape_22:0' shape=(3, 3) dtype=float32>, 'camera_translation': <tf.Tensor 'Reshape_35:0' shape=(3,) dtype=float32>, 'camera_image': <tf.Tensor 'decode_image/cond_jpeg

In [ ]:
!ls -l /tmp/tf3d_experiment/det_waymo_001

In [ ]:
# Check the generated event file and rename it here
%reload_ext tensorboard
%tensorboard --inspect --event_file=/tmp/tf3d_experiment/det_waymo_001/train/events.out.tfevents.1761938970.ea0e83a2b132.195.32893.v2

In [ ]:
%load_ext tensorboard
%tensorboard --inspect --event_file=/tmp/tf3d_experiment/det_waymo_001/eval_val_mesh/events.out.tfevents.1705992215.12310858bbc4.2957.21290.v2

In [ ]:
%load_ext tensorboard
%tensorboard --inspect --event_file=/tmp/tf3d_experiment/det_waymo_001/eval_val/events.out.tfevents.1638834720.71e8d45888ad.16694.8330768.v2


In [ ]:
import tensorflow.compat.v1 as tf1
from tensorflow.core.util import event_pb2
from tensorflow.python.lib.io import tf_record

tf1.compat.v1.disable_eager_execution()
tf1.compat.v1.disable_v2_behavior()

event_file_path = "/tmp/tf3d_experiment/det_waymo_001/eval_val_mesh/events.out.tfevents.1705992215.12310858bbc4.2957.21290.v2"

def my_summary_iterator(path):
    for r in tf_record.tf_record_iterator(path):
        yield event_pb2.Event.FromString(r)

writer = tf1.summary.FileWriter('/tmp/waymo_filters')

flg = 0

for event in my_summary_iterator(event_file_path):
    event_type = event.WhichOneof('what')
    if event_type != 'summary':
        writer.add_event(event)
    else:
        wall_time = event.wall_time
        step = event.step

filtered_values = [value for value in event.summary.value if 'val_predictions/1_' in value.tag]

summary = tf1.Summary(value=filtered_values)

filtered_event = tf1.Event(summary=summary, wall_time=wall_time, step=step)
writer.add_event(filtered_event)

writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/tf3d_experiment/det_waymo_001/train/